<a href="https://colab.research.google.com/github/stevenrdz/IR-SOES/blob/master/CS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from time import time
start_nb = time()

start = time()
import pandas as pd
from nltk.corpus import stopwords
import re
from nltk import word_tokenize

es_stops = set(stopwords.words('spanish'))

print('Importación de Librerias. %.2f segundos' % (time() - start))

Importación de Librerias. 0.00 segundos


In [ ]:
start = time()
data = pd.read_csv('//content/gdrive/My Drive/ultimate.csv',
                      sep=';', encoding='utf-8-sig')


print('Dataset generado. %.2f segundos' % (time() - start))
data.head()

Dataset generado. 0.27 segundos


,Id,Title,Body,Processed_Title,Processed_Body
0,286498,Debugear código Django utilizando Docker,<p>Les cuento.\r\nEstoy realizando un proyecto...,debugear código django utilizando docker,cuento realizando proyecto django utilizo pych...
1,226916,En el file.conf muestra Error usando Superviso...,<p>Al configurar el file.conf<br>\r\nruta : su...,muestra error usando supervisor digital ocean ...,configurar ruta sudo nano program facturaempre...
2,161561,Ejecutar navegador en pantalla completa al ini...,<p>Estoy realizando un <em>proyecto Domotico</...,ejecutar navegador pantalla completa iniciar l...,realizando proyecto domotico siguientes compon...
3,230560,Pagina web en blanco,<p>Buen dia \r\nNo se por que mi pagina web sa...,pagina web blanco,buen dia pagina web sale blanco alguien podria...
4,309146,Obtengo el error I/Choreographer: Skipped 110 ...,<p>Estoy trabajando en android studio intentan...,obtengo error skipped frames the application m...,trabajando android studio intentando hacer con...


In [ ]:
def preprocess(doc):
    code = re.compile('[¿?""(),:'']') # Eliminación de signos de puntuacion.
    doc = re.sub(code, '', doc)
    punt = re.compile('[./@_]') # Eliminación de signos de puntuacion.
    doc = re.sub(punt, ' ', doc)
    code = re.compile('<pre><code>[\s\S]*</code></pre>') # Eliminación de codigo.
    doc = re.sub(code, '', doc)
    tag = re.compile('<.*?>') # Eliminación de etiquetas.
    doc = re.sub(tag, '', doc)
    doc = doc.lower()  # Minuscula todo el texto.
    doc = word_tokenize(doc)  # Dividir en palabras.
    doc = [w for w in doc if not w in es_stops]  # Eliminar stopwords.
    # doc = [w for w in doc if w.isalpha()]  # Eliminar numbers and punctuation.
    return doc

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import string

titles_cosim = data['Processed_Title'].to_list()
id_index = data['Id'].to_list()
# Create the Document Term Matrix
vectorizer = CountVectorizer()

In [ ]:
def print_res(data, id):
    with pd.option_context('display.max_colwidth', 500):
        print(data.iloc[[id], [0,1]])

In [ ]:
# Create the Document Term Matrix
count_vectorizer = CountVectorizer()
sparse_matrix = count_vectorizer.fit_transform(titles_cosim)
doc_term_matrix = sparse_matrix.todense()
dtm = pd.DataFrame(doc_term_matrix,
                  columns=count_vectorizer.get_feature_names())

In [ ]:
# Create the Query Matrix
query = preprocess("Problemas con OSRM con R y Ubuntu 18.04")
print(query)
query_matrix = count_vectorizer.transform(query)
query_term_matrix = query_matrix.todense()
qtm = pd.DataFrame(query_term_matrix,
                  columns=count_vectorizer.get_feature_names())

['problemas', 'osrm', 'r', 'ubuntu']


In [ ]:
# Results and format
k = 5
cossim_matrix = cosine_similarity(qtm, dtm)
last = len(cossim_matrix)-1
exa = []
arg_sort = cossim_matrix[last].argsort()[-k:][::-1]
print(arg_sort)
value_sort = np.sort(cossim_matrix[last])[-k:][::-1]
print(value_sort)

[ 619  470 1048  923  456]
[0.70710678 0.70710678 0.70710678 0.70710678 0.70710678]


In [ ]:
for i in range(k):
    exa.append((arg_sort[i], value_sort[i]))

In [ ]:
for i in range(len(exa)):
  print_res(data, exa[i][0])
  print("sim: ",exa[i])
  print("-------------------------------------------------------")

         Id                            Title
619  265755  Java_Home no se setea en ubuntu
sim:  (619, 0.7071067811865475)
-------------------------------------------------------
         Id                                        Title
470  124058  ¿Cómo instalar php-mbregex en ubuntu 16.04?
sim:  (470, 0.7071067811865475)
-------------------------------------------------------
          Id                        Title
1048  303052  ¿Qué le sucede a mi Ubuntu?
sim:  (1048, 0.7071067811865475)
-------------------------------------------------------
         Id                Title
923  217036  Particionado Ubuntu
sim:  (923, 0.7071067811865475)
-------------------------------------------------------
         Id                        Title
456  198725  CronJob Ubuntu 16.04/php7.0
sim:  (456, 0.7071067811865475)
-------------------------------------------------------
